# Curriculum-Based AI Tutor

Objective:

To develop an AI tutor that answers student questions using a Retrieval-Augmented Generation (RAG) approach.

To enable semantic search over NCERT Class 8 Science content and provide accurate, grade-appropriate responses.

To evaluate model outputs using BLEU, ROUGE-L, and human review, ensuring factual correctness and transparency with source citations.

To deploy an interactive demo interface for students to engage with the AI tutor.

# Setting up Dependencies for RAG-based AI Tutor

In [ ]:

!pip install faiss-cpu==1.7.4 pdfplumber nltk rouge-score groq
!pip install numpy==1.26.4
!pip install "huggingface_hub>=0.27.0" "transformers>=4.44.2" "sentence-transformers>=2.2.2" "diffusers>=0.27.0" "gradio>=5.0.0" "peft>=0.10.0"



# Imports, Configuration, and Environment Setup

In [ ]:
# Cell 2: imports and config
import os, json, csv, time
from pathlib import Path
from typing import List, Dict, Tuple

import pdfplumber
import nltk
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')
nltk.download('punkt_tab')

# CONFIG - adjust as needed
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
# Either put a single PDF here OR a folder of PDFs (see extraction function)
SINGLE_PDF_PATH = DATA_DIR / "/content/science class 8.pdf"          # path if you have one file
#PDF_FOLDER = DATA_DIR / "class8_pdfs"                      # path if you have 13 separate pdfs (one per chapter)
JSONL_PATH = DATA_DIR / "class8_science.jsonl"
FAISS_INDEX_PATH = DATA_DIR / "faiss_index.index"
EMB_MODEL_NAME = "all-MiniLM-L6-v2"    # all-MiniLM-L6-v2 shorthand (SF model)
GROQ_MODEL = "llama3-8b-8192"             # default, change if needed and available to your key
K_RETRIEVE = 5
LOG_CSV = DATA_DIR / "interaction_log.csv"
EVAL_CSV = DATA_DIR / "evaluation.csv"


os.environ["GROQ_API_KEY"] = "gsk_g2Gu0krzPZOCvThl4MZ3WGdyb3FY9CQPJsxqFiNiuCA8i947EyX"
# Groq API key (must be set in env)
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    print("Warning: GROq_API_KEY not found in environment variables. "
          "Set it before calling generation (e.g. in Colab: %env GROQ_API_KEY=sk-...)")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# PDF Text Extraction with Chapter

In [ ]:

import re

def extract_from_single_pdf(pdf_path: Path) -> List[Dict]:
    """
    Extracts full text from one big PDF and attempts to detect chapter splits by heading keywords.
    Returns list of dicts: {"id","chapter","section","text"}.
    """
    items = []
    with pdfplumber.open(pdf_path) as pdf:
        pages = [p.extract_text() or "" for p in pdf.pages]
    text_all = "\n".join(pages)
    # Try split on common "Chapter" headings (case-insensitive)
    parts = re.split(r'\n\s*(CHAPTER|Chapter)\b.*', text_all)
    if len(parts) <= 2:
        # fallback: create page-level items
        for i, p in enumerate(pages):
            if p.strip():
                items.append({"id": f"page_{i}", "chapter": f"page_{i}", "section": f"page_{i}", "text": p.strip()})
    else:
        # naive chunks (some PDFs have inconsistent headings; you'll likely want to review)
        cur_id = 0
        for chunk in parts:
            txt = chunk.strip()
            if not txt:
                continue
            items.append({"id": f"chunk_{cur_id}", "chapter": f"chunk_{cur_id}", "section": f"chunk_{cur_id}", "text": txt})
            cur_id += 1
    return items

def extract_from_pdf_folder(folder: Path) -> List[Dict]:
    """
    Extracts text from each PDF file in folder. Uses filename as chapter name.
    """
    items = []
    pdf_files = sorted(list(folder.glob("*.pdf")))
    for pdf_path in pdf_files:
        chap_name = pdf_path.stem
        with pdfplumber.open(pdf_path) as pdf:
            pages = [p.extract_text() or "" for p in pdf.pages]
        text_all = "\n".join(pages).strip()
        # optional: split by "Chapter" inside each file too
        items.append({"id": chap_name, "chapter": chap_name, "section": chap_name, "text": text_all})
    return items

# Use whichever is present
docs = []
if SINGLE_PDF_PATH.exists():
    print("Extracting from single PDF:", SINGLE_PDF_PATH)
    docs = extract_from_single_pdf(SINGLE_PDF_PATH)
elif PDF_FOLDER.exists():
    print("Extracting from PDF folder:", PDF_FOLDER)
    docs = extract_from_pdf_folder(PDF_FOLDER)
else:
    print("No PDF found. Upload 'class8_science.pdf' to data/ or put chapter PDFs into data/class8_pdfs/")

print(f"Extracted {len(docs)} document chunks (candidate chapters/pages).")


Extracting from single PDF: /content/science class 8.pdf
Extracted 47 document chunks (candidate chapters/pages).


# Text Cleaning and Chunking into Passages with Metadata

In [ ]:

import re
import nltk

def clean_text(text: str) -> str:
    # Basic cleaning for NCERT-like text
    text = re.sub(r'\n+', ' ', text)  # Merge newlines
    text = re.sub(r'\b\d+\.\s+[A-Za-z ]{1,20}', '', text)  # Remove short numbered clues
    text = text.replace("1 C 2 O", "CO₂").replace("C 2 O", "CO₂")
    text = re.sub(r'\b[A-Z]\b', '', text)  # Remove single uppercase letters
    text = re.sub(r'\s+', ' ', text)  # Collapse extra spaces
    return text.strip()

def chunk_text_into_passages(text: str, max_tokens_or_words: int = 150) -> List[str]:
    text = clean_text(text)
    sents = nltk.sent_tokenize(text)
    cur = []
    cur_len = 0
    chunks = []
    for s in sents:
        words = s.split()
        if cur_len + len(words) <= max_tokens_or_words:
            cur.append(s)
            cur_len += len(words)
        else:
            chunks.append(" ".join(cur).strip())
            cur = [s]
            cur_len = len(words)
    if cur:
        chunks.append(" ".join(cur).strip())
    chunks = [c for c in chunks if len(c) > 20]  # Filter very short chunks
    return chunks

# Build passage list with metadata
passages = []
passage_id = 0
for d in docs:
    text = d.get("text", "")
    chapter = d.get("chapter", "unknown")
    chunks = chunk_text_into_passages(text, max_tokens_or_words=140)
    for i, c in enumerate(chunks):
        passages.append({
            "id": f"p_{passage_id}",
            "chapter": chapter,
            "section": d.get("section", ""),
            "para_index": i,
            "text": c
        })
        passage_id += 1

print("Created", len(passages), "passages.")
with open(JSONL_PATH, "w", encoding="utf-8") as f:
    for p in passages:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")
print("Saved cleaned corpus to", JSONL_PATH)




Created 594 passages.
Saved cleaned corpus to data/class8_science.jsonl
Loading embedding model: all-MiniLM-L6-v2
Embedding 594 passages...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

FAISS index built and saved to data/faiss_index.index
Saved passage metadata to data/passage_metadata.json


# Embedding Passages and Building FAISS Index

In [ ]:

from sentence_transformers import SentenceTransformer

print("Loading embedding model:", EMB_MODEL_NAME)
embed_model = SentenceTransformer(EMB_MODEL_NAME)

def embed_texts(model: SentenceTransformer, texts: List[str], batch_size: int = 64) -> np.ndarray:
    return np.array(
        model.encode(texts, show_progress_bar=True, convert_to_numpy=True, batch_size=batch_size),
        dtype="float32"
    )

texts = [p["text"] for p in passages]
print("Embedding", len(texts), "passages...")
embeddings = embed_texts(embed_model, texts)

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
faiss.write_index(index, str(FAISS_INDEX_PATH))
print("FAISS index built and saved to", FAISS_INDEX_PATH)

# Save metadata mapping
METADATA_PATH = DATA_DIR / "passage_metadata.json"
with open(METADATA_PATH, "w", encoding="utf-8") as f:
    json.dump(passages, f, ensure_ascii=False, indent=2)
print("Saved passage metadata to", METADATA_PATH)



# Verifying Cleaned Corpus Data

In [ ]:
import json

print("Previewing first 3 passages from", JSONL_PATH)
with open(JSONL_PATH, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(json.loads(line))

Previewing first 3 passages from data/class8_science.jsonl
{'id': 'p_0', 'chapter': 'chunk_0', 'section': 'chunk_0', 'para_index': 0, 'text': 'SSCCIIEENNCCEE VIII EXTBOOK FOR LASS 2018-19 ISBN 978-81-7450-812-6 First Edition January 2008 Magha 1929 ALL RIGHTS RESERVED Reprint Edition q No part of this publication may be reproduced, stored in a retrieval December 2008 Pausa 1930 system or transmitted, in any form or by any means, electronic, mechanical, photocopying, recording or otherwise without the prior January 2010 Magha 1931 permission of the publisher. November 2010 Kartika 1932 q This book is sold subject to the condition that it shall not, by way of January 2012 Magha 1933 trade, be lent, re-sold, hired out or otherwise disposed of without the publisher’s consent, in any form of binding or cover other than that in November 2012 Kartika 1934 which it is published.'}
{'id': 'p_1', 'chapter': 'chunk_0', 'section': 'chunk_0', 'para_index': 1, 'text': 'October 2013 Asvina 1935 q The

# Retrieval function (get top-k passages and return them)

In [ ]:


def load_index_and_meta(index_path: str, meta_path: str):
    idx = faiss.read_index(index_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    return idx, meta

def retrieve(query: str, top_k: int = 5) -> List[Dict]:
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    idx = faiss.read_index(str(FAISS_INDEX_PATH))
    _, I = idx.search(np.array(q_emb, dtype="float32"), top_k)
    hits = []
    for ii in I[0]:
        if ii < 0 or ii >= len(passages):
            continue
        hits.append(passages[ii])
    return hits

# Quick test
q = "What is photosynthesis?"
hits = retrieve(q, top_k=5)
print("Retrieved", len(hits), "snippets. Example snippet:\n", hits[0]["text"][:400])


Retrieved 5 snippets. Example snippet:
 These carry genes and a nuclear membrane are designated help in inheritance or transfer of as eukaryotic cells. All organisms other than bacteria and blue characters from the parents to the green algae are called eukaryotes. offspring. The chromosomes can be seen (eu : true; karyon: nucleus). only when the cell divides. CELL — STRUCTURE AND FUNCTIONS 95 2018-19 While observing the onion cells call


# Groq generation wrapper (uses only GROQ_API_KEY)

In [ ]:
!pip install groq

try:
    from groq import Client as GroqClient
    groq_client = GroqClient(api_key=os.getenv("GROQ_API_KEY"))
    def groq_generate(prompt: str, model: str = GROQ_MODEL, max_tokens: int = 512) -> str:

        if hasattr(groq_client, "chat"):
            resp = groq_client.chat.completions.create(model=model, messages=[{"role":"user","content":prompt}], max_tokens=max_tokens)
            # try to parse likely shapes:
            try:
                return resp.choices[0].message.content
            except Exception:
                # older shape
                return str(resp)
        elif hasattr(groq_client, "text"):
            resp = groq_client.text.create(model=model, input=prompt, max_tokens=max_tokens)
            try:
                # typical dict shape
                return resp["output"][0]["content"][0]["text"]
            except Exception:
                return str(resp)
        else:
            # fallback
            return str(groq_client.create(prompt))
except Exception as e:
    print("Groq client import error or not installed. Generation will fail until groq package available. Error:", e)
    def groq_generate(prompt: str, model: str = GROQ_MODEL, max_tokens: int = 512) -> str:
        raise RuntimeError("Groq client not available. Install 'groq' and set GROQ_API_KEY in env.")


# Prompt Construction and Question Answering Function

In [ ]:

def build_prompt(question: str, retrieved: List[Dict]) -> str:
    snippets = []
    for i, s in enumerate(retrieved):
        snippets.append(f"[{i+1}] (Chapter: {s.get('chapter')})\n{s.get('text')}\n")
    context_block = "\n\n".join(snippets)
    prompt = f"""
You are an AI tutor for NCERT Class 8 Science. Answer the student's question using ONLY the facts from the provided textbook snippets below.
If the question cannot be answered from the textbook content, reply exactly:

"I'm focused on Class 8 Science content. I don't have a textbook answer for that. Try re-phrasing your question to be within the textbook."

Provide:
1) A short, grade-appropriate answer (1–6 sentences).
2) A 'Sources' line listing the snippet indices you used, e.g. [1],[3].

Context snippets:
{context_block}

Question:
{question}

Answer:
"""
    return prompt

def answer_question(question: str, top_k: int = K_RETRIEVE) -> Dict:
    retrieved = retrieve(question, top_k=top_k)
    if not retrieved:
        return {"answer":"I'm focused on Class 8 Science; I don't have content to answer that question.", "retrieved": []}
    prompt = build_prompt(question, retrieved)
    gen = groq_generate(prompt, model=GROQ_MODEL, max_tokens=512)
    # Logging
    with open(LOG_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([time.time(), question, gen, "|".join([r["id"] for r in retrieved])])
    return {"answer": gen, "retrieved": retrieved}



# QA System Evaluation with BLEU, ROUGE-L & BERTScore




In [ ]:
!pip install bert-score

# Evaluation (with normalization + smoothing + multiple golds + source cleaning)
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bert_score import score

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
smooth = SmoothingFunction().method1

def normalize_text(text: str) -> str:
    """Lowercase, remove punctuation/stopwords, lemmatize"""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)   # keep only alphanum + spaces
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

def clean_generated(gen: str) -> str:
    """Remove trailing sources or irrelevant parts from generated answer"""
    if "Sources:" in gen:
        gen = gen.split("Sources:")[0]
    return gen.strip()

def evaluate(test_pairs: List[Dict], top_k: int = K_RETRIEVE) -> List[Dict]:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rows = []

    for t in test_pairs:
        q = t["question"]
        golds = t["gold_answer"]

        # allow both single-string and multi-reference
        if isinstance(golds, str):
            golds = [golds]

        out = answer_question(q, top_k=top_k)
        gen = clean_generated(out["answer"])

        # normalized
        gen_norm = normalize_text(gen)
        golds_norm = [normalize_text(g) for g in golds]

        # BLEU with smoothing
        try:
            bleu = sentence_bleu(
                [nltk.word_tokenize(g) for g in golds_norm],
                nltk.word_tokenize(gen_norm),
                smoothing_function=smooth
            )
        except Exception:
            bleu = 0.0

        # ROUGE-L (take best among multiple golds)
        try:
            rougeL = max([scorer.score(g, gen_norm)["rougeL"].fmeasure for g in golds_norm])
        except Exception:
            rougeL = 0.0

        # BERTScore (semantic similarity, pick best gold)
        try:
            bert_f1 = max([score([gen], [g], lang="en", verbose=False)[2].item() for g in golds])
        except Exception:
            bert_f1 = 0.0

        rows.append({
            "question": q,
            "gold": golds[0] if len(golds)==1 else golds,  # keep first or list
            "gen": gen,
            "bleu": bleu,
            "rougeL": rougeL,
            "bert_f1": bert_f1
        })

    # Save
    df = pd.DataFrame(rows)
    df.to_csv(EVAL_CSV, index=False)
    print("Saved evaluation to", EVAL_CSV)

    return rows
test_pairs = [
    {
        "question": "What is crop rotation?",
        "gold_answer": [
            "Crop rotation is the practice of growing different crops in succession on the same field to maintain soil fertility.",
            "Crop rotation means planting different crops alternately in the same field to prevent soil nutrients from being exhausted.",
            "It is the method of cultivating different crops one after another on the same land to keep the soil fertile."
        ]
    },
    {
        "question": "Name two methods of irrigation used in agriculture.",
        "gold_answer": [
            "Two common methods of irrigation are sprinkler system and drip irrigation.",
            "Irrigation can be done by sprinkler method or drip method.",
            "Two important irrigation techniques are the sprinkler and the drip system."
        ]
    },
    {
        "question": "What are microorganisms?",
        "gold_answer": [
            "Microorganisms are tiny living organisms, such as bacteria, fungi, protozoa and some algae, which can only be seen under a microscope.",
            "Very small organisms like bacteria, protozoa, some fungi and algae that cannot be seen without a microscope are called microorganisms.",
            "Microorganisms are minute organisms that are visible only under a microscope."
        ]
    },
    {
        "question": "Name one disease caused by bacteria and one caused by protozoa.",
        "gold_answer": [
            "Tuberculosis is caused by bacteria, while malaria is caused by protozoa.",
            "A bacterial disease is tuberculosis, and a protozoan disease is malaria.",
            "Tuberculosis is an example of a bacterial disease; malaria is an example of a protozoan disease."
        ]
    },
    {
        "question": "What are synthetic fibres? Give two examples.",
        "gold_answer": [
            "Synthetic fibres are man-made fibres prepared from chemicals. Examples include nylon and polyester.",
            "Fibres made by humans from chemicals are called synthetic fibres, such as nylon and polyester.",
            "Synthetic fibres are artificial fibres, for example polyester and nylon."
        ]
    },
    {
        "question": "Write one advantage and one disadvantage of plastics.",
        "gold_answer": [
            "Plastics are durable and light in weight, but they are non-biodegradable and cause pollution.",
            "Plastics are strong and light, but they do not decompose easily and pollute the environment.",
            "An advantage of plastic is durability; a disadvantage is that it is non-biodegradable."
        ]
    },
    {
        "question": "Name two metals and two non-metals.",
        "gold_answer": [
            "Examples of metals are iron and copper, while examples of non-metals are sulphur and carbon.",
            "Iron and copper are metals; sulphur and carbon are non-metals.",
            "Two metals are iron and copper; two non-metals are carbon and sulphur."
        ]
    },
    {
        "question": "What are fossil fuels?",
        "gold_answer": [
            "Fossil fuels are natural fuels like coal, petroleum, and natural gas formed from the remains of dead plants and animals over millions of years.",
            "Coal, petroleum, and natural gas are fossil fuels formed from ancient plant and animal remains.",
            "Fuels such as coal, oil, and gas formed from buried dead organisms are called fossil fuels."
        ]
    },
    {
        "question": "Why is petroleum called black gold?",
        "gold_answer": [
            "Petroleum is called black gold because of its black colour and high economic value.",
            "Due to its dark colour and great commercial importance, petroleum is known as black gold.",
            "Petroleum is referred to as black gold since it is black in colour and extremely valuable."
        ]
    },
    {
        "question": "What is combustion?",
        "gold_answer": [
            "Combustion is a chemical process in which a substance reacts with oxygen to release heat and light.",
            "The burning of a substance with oxygen to give out heat and light is called combustion.",
            "Combustion is the process of burning in which a material reacts with oxygen producing heat and light."
        ]
    },
    {
        "question": "What is the SI unit of force?",
        "gold_answer": [
            "The SI unit of force is newton (N).",
            "Force is measured in newtons in the SI system.",
            "The standard SI unit for force is called a newton."
        ]
    },
    {
        "question": "Define pressure.",
        "gold_answer": [
            "Pressure is the force acting per unit area.",
            "When a force is applied per unit area, it is called pressure.",
            "Pressure is defined as force divided by area."
        ]
    },
    {
        "question": "What is the force of friction?",
        "gold_answer": [
            "Friction is the opposing force that comes into play when one surface moves or tends to move over another surface.",
            "The contact force that resists motion between two surfaces is called friction.",
            "Friction is the force that opposes relative motion between two surfaces in contact."
        ]
    },
    {
        "question": "What is a cell?",
        "gold_answer": [
            "A cell is the structural and functional unit of life.",
            "The basic structural and functional unit of all living organisms is called a cell.",
            "Cells are the smallest units of life, making up the structure and function of organisms."
        ]
    },
    {
        "question": "What is the function of haemoglobin?",
        "gold_answer": [
            "Haemoglobin in red blood cells helps transport oxygen from the lungs to body tissues.",
            "The function of haemoglobin is to carry oxygen in the blood.",
            "Haemoglobin transports oxygen from the lungs to all body parts."
        ]
    },
    {
        "question": "What is photosynthesis?",
        "gold_answer": [
            "Photosynthesis is the process by which green plants prepare their own food from carbon dioxide and water using sunlight in the presence of chlorophyll.",
            "Green plants make food by using water and carbon dioxide in sunlight; this is called photosynthesis.",
            "Photosynthesis is the process where plants prepare food in the presence of chlorophyll and sunlight."
        ]
    },
    {
        "question": "Name two methods of water purification used at home.",
        "gold_answer": [
            "Two methods of water purification are boiling and filtration.",
            "Water can be purified at home by boiling or by filtering.",
            "Boiling and filtration are common methods to make water safe for drinking at home."
        ]
    },
    {
        "question": "What is pollution?",
        "gold_answer": [
            "Pollution is the undesirable change in the physical, chemical, or biological characteristics of air, water, or soil.",
            "When air, water, or soil is contaminated, it is called pollution.",
            "Pollution means harmful changes in the environment caused by impurities."
        ]
    },
    {
        "question": "What is the ozone layer?",
        "gold_answer": [
            "The ozone layer protects the Earth from harmful ultraviolet rays of the Sun.",
            "A layer of ozone gas in the atmosphere that blocks harmful UV radiation is called the ozone layer.",
            "The ozone layer shields Earth by absorbing ultraviolet rays from the Sun."
        ]
    },
    {
        "question": "What is global warming?",
        "gold_answer": [
            "Global warming is the gradual increase in the Earth’s average temperature due to excessive greenhouse gases in the atmosphere.",
            "The rise in Earth’s average temperature caused by greenhouse gases is known as global warming.",
            "Global warming means the slow increase in Earth’s temperature due to carbon dioxide and other gases."
        ]
    }
]


#  Run evaluation
rows = evaluate(test_pairs)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are ne

Saved evaluation to data/evaluation.csv


# Show saved files

In [ ]:
# Cell 10: show saved files and next steps
print("Files created in data/:")
for p in DATA_DIR.iterdir():
    print("-", p.name)


Files created in data/:
